`Task`: Editing to make on the Nokia and Huawei SLA Report
- Bandwidth from 1GBits
- Filter out Enterprise Links – these are links that has   LL_LM , LL_CO , INT_LM , INT_CO 
- Filter out Zero Reachability 
- Filter out all the IPDCN Links
- Filter out all the Ericsson 
- Comparing dublicates
- Add more conditions for 70-80, 80-90, and 90 and above exceptions

## Import

In [1]:
# Use data without excel formulas
file_path = 'CTN Nokia Created.xlsx' # Nokia
# file_path = 'CTN Huawei Created.xlsx' # Huawei
# file_path = 'Book1.xlsx' # Huawei


In [2]:
# Libraries
import pandas as pd
import re
import math
import numpy as np
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [3]:
def convert_and_multiply(value):
    if not isinstance(value, str):
        return 0
    if value.endswith('G'):
        num_value=float(value[:-2]) * 1000
    elif value.endswith('M'):
        num_value=float(value[:-2]) * 1
    elif value.endswith('T'):
        num_value=float(value[:-2]) * 1000000
    else:
        return value
    return int(num_value)

In [4]:
df = pd.read_excel(file_path, sheet_name=0, header=0, index_col=0)

In [5]:
def columns_to_2dp(df=df):
    
    # Assuming your DataFrame is named 'df' and you want to convert specific columns
    columns_to_convert = ['Reachability', 'Utilization', 'Utilization.1',
                          'Utilization.2', 'Utilization.3', 'Utilization.4', 'Utilization.5', 'Utilization.6']

    # Fill NA or blanks with zero
    df[columns_to_convert] = df[columns_to_convert].fillna(0)

    # Define a custom rounding function
    def round_up(x):
        if x % 1 >= 0.005:
            return np.ceil(x * 100) / 100 
        else:
            return np.floor(x)

    # Round up float values to the nearest whole number based on the custom rounding function
    df[columns_to_convert] = df[columns_to_convert].applymap(round_up)

#     # Convert rounded float columns to integers
#     df[columns_to_convert] = df[columns_to_convert].astype(np.int64)
    
    return df


In [6]:
def filter_task(df=df):
    # load excel data
    
    '''Convert to int ['Reachability', 'Utilization', 'Utilization.1',
    'Utilization.2', 'Utilization.3', 'Utilization.4', 'Utilization.5', 'Utilization.6']''' 
    columns_to_2dp()
    #Display shape
    print('Initial Total per region:',region_counts := df['Region'].value_counts())
    print("initial shape:", df.shape)
    print("converted dtypes:", df.dtypes)
    
    # Filter Bandwidth >= 1GBits
    df_bandwidth_blank=df[df['Bandwidth'].isnull()]
    df_bandwidth_blank.head()
    df = df.dropna(subset=['Bandwidth'])
    pattern = r'(\d+(?:\.\d+)? [GMT])'
    df['Bandwidth_extracted'] = df['Bandwidth'].str.extract(pattern)    
    df['Bandwidth_extracted'] = df['Bandwidth_extracted'].apply(convert_and_multiply).astype(int)
    df = df[df['Bandwidth_extracted']>= 1000]
    
    print("1st step:- After Bandwidth filter:", df.shape)
    print("Shape of blank bandwidth data:", df_bandwidth_blank.shape)
    df_bandwidth_blank = df_bandwidth_blank[df_bandwidth_blank.iloc[:, -1] >= 4]
    print('How many blank bandwidth that meets the utilization rule:',
          df_bandwidth_blanc :=df_bandwidth_blank['Region'].value_counts())
    
    # Filter out Enterprise Links (LL_LM, LL_CO, INT_LM, INT_CO)
    names_to_remove = ['LL_LM', 'LL_CO', 'INT_LM', 'INT_CO', 'to-Ericsson-6672']
    df = df[~df['Interface'].str.endswith(tuple(names_to_remove))]

    print("2nd step:- After Enterprise filter:", df.shape)
    
    # Filter out zero reachability
    df_reachability_blank = df[df['Reachability'].isnull()] #Saving reachability with blank data
    df = df[df['Reachability'] > 0] # removes zeros and below
    
    print("3rd step:- After zero reachability filter:", df.shape)
    
    # Filter out all the IPDCN Links
    df = df[~df['Interface'].str.endswith('IPDCN')]
    df = df[~df['Interface'].str.contains('IPDCN')]
    
    print("4th step:- After IPDCN Links removed:", df.shape)
    
    #Filter out all the Ericsson Links
    df = df[~df['Interface'].str.contains('to_Ericsson|to-Ericsson', case=False)]
    
    # Comparing dublicates
    df['Interface_new'] = df['Interface'].str.extract(r'^([^_]+)')
    df['Total_util'] = df.filter(like='Utilization').sum(axis=1)
    # Sort the DataFrame by 'total_util' column in descending order
    df.sort_values('Total_util', ascending=False, inplace=True)
    # Drop duplicates based on 'interface_new' column, keeping the first occurrence (highest total_util)
    df.drop_duplicates(subset='Interface_new', keep='first', inplace=True)

    
    print("Final step:- After all conditions are met:", df.shape)
    
    # Count by Region
    display('After conditions are met:- Total per region:',region_counts := df['Region'].value_counts())
    
    # Count the number of NaN values in the column
    nan_count = df['Region'].isna().sum()

    # Count the number of blank values in the column
    blank_count = (df['Region'] == '').sum()
    
    print('How many nan,blank are in Region Column:', nan_count, blank_count)
    
    df.drop(['Bandwidth_extracted', 'Interface_new', 'Total_util'], axis=1, inplace=True)
        
    # Filter out data in column 11 with value 4 and above
    df_filtered90andabove = df[df['90% and above'] >= 4]
    df_filtered80_90 = df[df['80% to 90%'] >= 4]
    df_filtered70_80 = df[df['70% to 80%'] >= 4]
    
    # blank
#     print("Shape of blank reachability data:", df_reachability_blank.shape)
#     df_filtered_blank = df_reachability_blank[df_reachability_blank.iloc[:, -2] >= 4]
#     print('How many blanks per region that meets rule:', df_filtered_blanc :=df_filtered_blank['Region'].value_counts())
#     print("shape of filtered data:", df_filtered.shape)
    
#     # Count the number of NaN values in the column
#     nan_count_f = df_filtered['Region'].isna().sum()
#     # Count the number of blank values in the column
#     blank_count_f = (df_filtered['Region'] == '').sum()
    
#     print('How many nan,blank are in Region Column in Filtered:', nan_count_f, blank_count_f)
    
    # Group by region
    print('How many are above 4 in the 90 and above condition:',
          region_counts := df_filtered90andabove['Region'].value_counts(), df_filtered90andabove.shape)
    print('-----')
    print('How many are above 4 in the 80 to 90 condition:',
          region_counts := df_filtered80_90['Region'].value_counts(), df_filtered80_90.shape)
    print('-----')
    print('How many are above 4 in the 70 to 80 condition:',
          region_counts := df_filtered70_80['Region'].value_counts(), df_filtered70_80.shape)
    print('-----')
    
    

    # Merging the filted data.

    # Concatenate the dataframes
    df_filtered = pd.concat([df_filtered90andabove, df_filtered80_90, df_filtered70_80])

    # Remove duplicates based on all columns
    df_filtered = df_filtered.drop_duplicates()

    # Sort by the '90% and above' column in descending order
    df_filtered = df_filtered.sort_values(by=['90% and above', '80% to 90%', '70% to 80%'],
                                          ascending=[False, False, False])



    
#     df.drop(['Bandwidth_extracted', 'Interface_new', 'Total_util'], axis=1, inplace=True)
#     df_filtered.drop(['Bandwidth_extracted', 'Interface_new', 'Total_util'], axis=1, inplace=True)


    df.reset_index(drop=True, inplace=True)
    df_filtered.reset_index(drop=True, inplace=True)
    df.index += 1
    df_filtered.index += 1
    df.index.name = '#'
    df_filtered.index.name = '#'
        
    save_df=df.to_excel('done_'+file_path, sheet_name=file_path)
    save_filtered=df_filtered.to_excel('filtered_' + file_path, sheet_name=file_path)
    return save_df, save_filtered


In [8]:
filter_task()

Initial Total per region: Asaba                             8931
Kano                              5916
Enugu                             2229
PHC                                  9
T2632_CHINEDU EZENYILLI_SASR_1       8
Abuja                                7
unknown                              1
ENUGU                                1
Name: Region, dtype: int64
initial shape: (52085, 14)
converted dtypes: Region            object
Interface         object
Bandwidth         object
Reachability     float64
Utilization      float64
Utilization.1    float64
Utilization.2    float64
Utilization.3    float64
Utilization.4    float64
Utilization.5    float64
Utilization.6    float64
90% and above      int64
80% to 90%         int64
70% to 80%         int64
dtype: object
1st step:- After Bandwidth filter: (16104, 15)
Shape of blank bandwidth data: (35788, 14)
How many blank bandwidth that meets the utilization rule: Series([], Name: Region, dtype: int64)
2nd step:- After Enterprise filter: (14

C:\Users\admin\AppData\Local\Temp\ipykernel_43268\59886722.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bandwidth_extracted'] = df['Bandwidth'].str.extract(pattern)
C:\Users\admin\AppData\Local\Temp\ipykernel_43268\59886722.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Bandwidth_extracted'] = df['Bandwidth_extracted'].apply(convert_and_multiply).astype(int)


'After conditions are met:- Total per region:'

Asaba    548
Kano     394
Enugu    146
Name: Region, dtype: int64

How many nan,blank are in Region Column: 0 0
How many are above 4 in the 90 and above condition: Asaba    6
Kano     1
Name: Region, dtype: int64 (7, 14)
-----
How many are above 4 in the 80 to 90 condition: Kano    1
Name: Region, dtype: int64 (1, 14)
-----
How many are above 4 in the 70 to 80 condition: Kano     4
Enugu    2
Asaba    2
Name: Region, dtype: int64 (8, 14)
-----


(None, None)

In [9]:
# # Checking if any of this words are found.

# # Define the patterns and keywords
# patterns = ['LL_LM$', 'LL_CO$', 'INT_LM$', 'INT_CO$', 'IPDCN$']
# keyword = 'Ericsson'

# # Create a boolean mask to filter the rows
# mask = df['Column'].str.contains('|'.join(patterns), case=False, regex=True) & \
#        df['Column'].str.contains(keyword, case=False, regex=False)

# # Apply the mask to the DataFrame
# filtered_df = df[mask]
